# Construction of a public dataset of Blood Bowl matches played on FUMBBL.com

This blogpost is about **Blood Bowl**, a boardgame I started playing last year. The goal of this blog post is to use Python API and HTML scraping to fetch online Blood Bowl match outcome data, and to create a structured dataset ready for analysis and visualization. This blogpost is written as a Jupyter notebook containing Python code, and is fully reproducible. The idea is to make Blood Bowl data analysis accessible to others. Using open source tooling reduces the barriers for others to build on other people’s work.


In [1]:
import random
import time
import os

from isoweek import Week

import requests # API library

import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Blood Bowl online: FUMBBL 

The **FUMBBL** website (https://fumbbl.com) is one big pile of data. From coach pages, with their teams, to team rosters, with players, and match histories. It's all there.

To obtain **FUMBBL** data, we need to fetch it match by match, team by team. To do so, the site creator Christer Kaivo-oja, from Sweden, has made an API that allows us to easily fetch data. 

# Using Python requests package to fetch data
We use the [Python **Requests** library](https://docs.python-requests.org/en/latest/) to make the API call over HTTPS and obtain the response from the FUMBLL server. The response is in the JSON format, a [light-weight data-interchange format](https://www.json.org/json-en.html) which is both easy to read and write for humans, and easy to parse and generate by computers. So this makes it a natural choice for an API.
The full documentation of the API can be found at (https://fumbbl.com/apidoc/).


# Data structure

We go with a flat data frame with **rows for each match**, and columns for the various variables associated with each match.
These would include:

* Coach ids
* Team races
* Team ids
* Date of the match
* Outcome (Touchdowns of both teams)

With this basic structure, we can add as many match related variables in the future, keeping the basic structure (each row is a match) unchanged.

So lets get the match data!



# Step 1: API scraping the match data: df_matches

So we are mostly interested in the current ruleset, this is `BB2020`. This ruleset became available in **FUMBBL** at september 1st 2021, and two months later, some 5000 games have been played. We also want to compare with the previous ruleset, where we have much more data available. 
The dataset start with match `4216258` played on august 1st, 2020. This covers roughly 12 months of `BB2016` ruleset matches, after that it switches to predominantly `BB2020` matches.

We collect match data by looping over `match_id`. We store the full JSON file on disk, so we avoid repeat API calls in the future.

**VERY IMPORTANT: We do not want to overload the **FUMBBL** server, so we make only three API requests per second. In this way, the server load is hardly affected and it can continue functioning properly for all the Blood Bowl coaches playing their daily games!**

In [2]:
%run write_json_file.py

In [3]:
# PM split out fetching JSON and transforming to pandas
df_matches = pd.DataFrame(columns=['match_id', 'replay_id', 'tournament_id', 'match_date', 'match_time',  'match_conceded',
    'team1_id', 'team1_coach_id', 'team1_roster_id', 'team1_race_name', 'team1_value', 'team1_cas_bh', 'team1_cas_si', 'team1_cas_rip',
    'team2_id', 'team2_coach_id', 'team2_roster_id', 'team2_race_name', 'team2_value', 'team2_cas_bh', 'team2_cas_si', 'team2_cas_rip',
    'team1_score', 'team2_score'])

target = 'raw/df_matches_' + time.strftime("%Y%m%d_%H%M%S") + '.h5'
print(target)

end_match = 4460944 
begin_match = 4429319
n_matches = end_match - begin_match
full_run = 0
print("matches to grab:")
print(n_matches)

if(full_run):
    for i in range(n_matches):
        match_id = end_match - i
        api_string = "https://fumbbl.com/api/match/get/" + str(match_id)

        match = requests.get(api_string)
        match = match.json()

        dirname = "raw/match_html_files/" + str(match_id)[0:4]
        if not os.path.exists(dirname):
            os.makedirs(dirname)

        fname_string = dirname + "/match_" + str(match_id) + ".json"

        write_json_file(match, fname_string)

        if match: # fix for matches that do not exist
            match_id = match['id']
            replay_id = match['replayId']
            tournament_id = match['tournamentId'] # key to tournament table
            match_date = match['date']
            match_time = match['time']
            match_conceded = match['conceded']
            team1_id = match['team1']['id']
            team2_id = match['team2']['id']
            # touchdowns
            team1_score = match['team1']['score']
            team2_score = match['team2']['score']  
            # casualties
            team1_cas_bh = match['team1']['casualties']['bh']
            team1_cas_si = match['team1']['casualties']['si']
            team1_cas_rip = match['team1']['casualties']['rip']
            team2_cas_bh = match['team2']['casualties']['bh']
            team2_cas_si = match['team2']['casualties']['si']
            team2_cas_rip = match['team2']['casualties']['rip']
            # other
            team1_roster_id = match['team1']['roster']['id']
            team2_roster_id = match['team2']['roster']['id']            
            team1_coach_id = match['team1']['coach']['id']
            team2_coach_id = match['team2']['coach']['id']
            team1_race_name = match['team1']['roster']['name'] 
            team2_race_name = match['team2']['roster']['name'] 
            team1_value = match['team1']['teamValue']
            team2_value = match['team2']['teamValue']
            #print(match_id)     
            df_matches.loc[i] = [match_id, replay_id, tournament_id, match_date, match_time, match_conceded,
                team1_id, team1_coach_id, team1_roster_id, team1_race_name, team1_value, team1_cas_bh, team1_cas_si, team1_cas_rip, 
                team2_id, team2_coach_id, team2_roster_id, team2_race_name, team2_value, team2_cas_bh, team2_cas_si, team2_cas_rip, 
                team1_score, team2_score]
        else:
            # empty data for this match, create empty row
            match_id = int(end_match - i)
            df_matches.loc[i] = np.repeat([np.NaN], 24, axis=0)
            df_matches.loc[i]['match_id'] = int(match_id)
        if i % 100 == 0: 
            # write tmp data as hdf5 file
            print(i, end='')
            print(".", end='')
            df_matches.to_hdf(target, key='df_matches', mode='w')

    # write data as hdf5 file
    df_matches.to_hdf(target, key='df_matches', mode='w')
else:
    # read from hdf5 file
    df_matches1 = pd.read_hdf('raw/df_matches_20220310_155600.h5') # 4216259 - 4221258
    df_matches2 = pd.read_hdf('raw/df_matches_20220316_180506.h5') # 4221259 - 4221820
    df_matches3 = pd.read_hdf('raw/df_matches_20220311_084424.h5') # 4221821 - 4271820
    df_matches4 = pd.read_hdf('raw/df_matches_20220311_231408.h5') # 4271821 - 4321820
    df_matches5 = pd.read_hdf('raw/df_matches_20220312_083221.h5') # 4321821 - 4370543
    df_matches6 = pd.read_hdf('raw/df_matches_20220606_225206.h5') # 4374338 - 4386470
    df_matches7 = pd.read_hdf('raw/df_matches_20220607_060907.h5') # 4370544 - 4374337
    df_matches8 = pd.read_hdf('raw/df_matches_20230108_205542.h5') # 4429319 - 4386470
    df_matches9 = pd.read_hdf('raw/df_matches_20230523_222012.h5') # 4460944 - 4429319
    df_matches = pd.concat([df_matches1, df_matches2, df_matches3, 
        df_matches4, df_matches5, df_matches6, df_matches7, df_matches8, df_matches9], ignore_index=True)

df_matches.shape
df_matches = df_matches.sort_values(by=['match_id']).reset_index(drop=True)


raw/df_matches_20230524_191844.h5
matches to grab:
31625


In [4]:
df_matches.shape

(244652, 24)

In [5]:
df_matches.query('match_id == 4460944')

,match_id,replay_id,tournament_id,match_date,match_time,match_conceded,team1_id,team1_coach_id,team1_roster_id,team1_race_name,team1_value,team1_cas_bh,team1_cas_si,team1_cas_rip,team2_id,team2_coach_id,team2_roster_id,team2_race_name,team2_value,team2_cas_bh,team2_cas_si,team2_cas_rip,team1_score,team2_score
244651,4460944,1622510,0,2023-05-23,04:52:59,None,1116389,107478,4965,Imperial Nobility,1450k,0,0,0,1122664,258292,4956,Black Orc,1740k,0,2,0,0,2


## Data prep: fixing the datatypes, creating derived variables

Since we manually filled the `pandas` DataFrame, most of the columns are now of `object` datatype.
We need to change this to be able to work properly with the data, as well as store it properly.
Here I convert each column manually, however I later found out about `DataFrame.infer_objects()`, that can detect the proper dtype automatically.
This I will try next time.

In [6]:
# convert object dtype columns to proper pandas dtypes datetime and numeric
df_matches['match_date'] = pd.to_datetime(df_matches.match_date) # Datetime object
df_matches['match_id'] = pd.to_numeric(df_matches.match_id) 
df_matches['replay_id'] = pd.to_numeric(df_matches.replay_id) 
df_matches['tournament_id'] = pd.to_numeric(df_matches.tournament_id) 
df_matches['team1_id'] = pd.to_numeric(df_matches.team1_id) 
df_matches['team1_coach_id'] = pd.to_numeric(df_matches.team1_coach_id) 
df_matches['team1_roster_id'] = pd.to_numeric(df_matches.team1_roster_id) 
df_matches['team2_id'] = pd.to_numeric(df_matches.team2_id) 
df_matches['team2_coach_id'] = pd.to_numeric(df_matches.team2_coach_id) 
df_matches['team2_roster_id'] = pd.to_numeric(df_matches.team2_roster_id) 
df_matches['team1_score'] = pd.to_numeric(df_matches.team1_score) 
df_matches['team2_score'] = pd.to_numeric(df_matches.team2_score) 

# calculate match score difference
df_matches['team1_win'] = np.sign(df_matches['team1_score'] - df_matches['team2_score'])
df_matches['team2_win'] = np.sign(df_matches['team2_score'] - df_matches['team1_score'])

# mirror match
df_matches['mirror_match'] = 0
df_matches.loc[df_matches['team1_race_name'] == df_matches['team2_race_name'], 'mirror_match'] = 1


In [7]:
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244652 entries, 0 to 244651
Data columns (total 27 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   match_id         244652 non-null  int64         
 1   replay_id        244612 non-null  float64       
 2   tournament_id    244612 non-null  float64       
 3   match_date       244612 non-null  datetime64[ns]
 4   match_time       244612 non-null  object        
 5   match_conceded   244612 non-null  object        
 6   team1_id         244612 non-null  float64       
 7   team1_coach_id   244612 non-null  float64       
 8   team1_roster_id  244612 non-null  float64       
 9   team1_race_name  244612 non-null  object        
 10  team1_value      244612 non-null  object        
 11  team1_cas_bh     244612 non-null  object        
 12  team1_cas_si     244612 non-null  object        
 13  team1_cas_rip    244612 non-null  object        
 14  team2_id         244

In [8]:
df_matches['team1_cas_bh'] = pd.to_numeric(df_matches.team1_cas_bh) 
df_matches['team1_cas_si'] = pd.to_numeric(df_matches.team1_cas_si) 
df_matches['team1_cas_rip'] = pd.to_numeric(df_matches.team1_cas_rip) 
# add total CAS
df_matches['team1_cas'] = df_matches['team1_cas_bh'] + df_matches['team1_cas_si'] + df_matches['team1_cas_rip']


df_matches['team2_cas_bh'] = pd.to_numeric(df_matches.team2_cas_bh) 
df_matches['team2_cas_si'] = pd.to_numeric(df_matches.team2_cas_si) 
df_matches['team2_cas_rip'] = pd.to_numeric(df_matches.team2_cas_rip) 
# add total CAS
df_matches['team2_cas'] = df_matches['team2_cas_bh'] + df_matches['team2_cas_si'] + df_matches['team2_cas_rip']

In [9]:
# mirror matches
df_matches.query('mirror_match == 1').shape

(8870, 29)

## Dataprep: transforming the team values

In Blood Bowl, teams can develop themselves over the course of multiple matches. The winnings of each match can be spend on buying new, stronger players, or replace the players that ended up getting injured or even killed. In addition, players receive so-called *star points* for important events, such as scoring, or inflicting a casualty on the opponent. Therefore, a balancing mechanism is needed when a newly created "rookie" team is facing a highly developed opposing team with lots of extra skills and strong players. 

Blood Bowl solves this by calculating for both teams their **Current team value**.
The **Team value difference** for a match determines the amount of gold that the weaker team can use to buy so-called **inducements**.
These inducements are temporary, and can consists of a famous "star player" who joins the team just for this match. Another popular option is to hire a wizard that can be used to turn one of the opposing players into a frog.

It is well known that the win rates of the teams depend on how developed a team is. For example, Amazons are thought to be strongest at low team value, as they already start out with lots of *block* and *dodge* skills, whereas a Chaos team start out with almost no skills.
So if we compare win rates, we would like take into account the current team value. 
Now as this can differ between the two teams in a match up, I reasoned that the highest team value is most informative about the average strength level of both teams, because of the inducement mechanism described above. (In the next step, we will add information on inducements)

In the dataset, we have for each match the current team values of both teams as a text string. 
We transform the text string `1100k` into an integer number `1100`, so that we can calculated the difference as `tv_diff`, and pick for each match the maximum team value and store it as `tv_match`. Finally, we create a team value bin `tv_bin` to be able to compare win rates for binned groups of matches where races have comparable team strength / team development.


In [10]:
# convert team value 1100k to 1100 integer and and above / below median (= low / high TV)
df_matches['team1_value'] = df_matches['team1_value'].str.replace('k$', '')
df_matches['team1_value'] = df_matches['team1_value'].fillna(0).astype(np.int64)

df_matches['team2_value'] = df_matches['team2_value'].str.replace('k$', '')
df_matches['team2_value'] = df_matches['team2_value'].fillna(0).astype(np.int64)

df_matches['tv_diff'] = np.abs(df_matches['team2_value'] - df_matches['team1_value'])
df_matches['tv_diff2'] = df_matches['team2_value'] - df_matches['team1_value']

df_matches['tv_match'] = df_matches[["team1_value", "team2_value"]].max(axis=1)

df_matches['tv_bin'] = pd.cut(df_matches['tv_match'], 
    bins = [0, 950, 1250,1550, 1850, float("inf")], 
    labels=['< 950K', '1.1M', '1.4M', '1.7M', '> 1850K']
)

df_matches['tv_bin2'] = pd.cut(df_matches['tv_match'], 
    bins = [0, 950, 1050, 1150, 1250, 1350, 1450, 1550, float("inf")], 
    labels=['< 950K', '1.0M', '1.1M', '1.2M',  '1.3M', '1.4M', '1.5M', '> 1550K']
)


## Dropping empty matches

Some match_id's do not have match information attached to them, presumably these matches were not played or some real life event interfered. These match_ids are dropped from the dataset to get rid of the NAs in all the columns.

In [11]:
len(df_matches)
df_matches = df_matches.dropna(subset=['match_date'])

## Dataprep: getting the dates right

To see time trends, its useful to aggregate the data by week. For this we add `week_number` for each date, and from this week number, we convert back to a date to get a `week_date`. This last part is useful for plotting with `plotnine`, as this treats dates in a special way
We use the ISO definition of week, this has some unexpected behavior near the beginning / end of each year. 

The data starts in week 36 (september) of 2020, and stops halfway march 2022.


In [12]:
df_matches['week_number'] = df_matches['match_date'].dt.isocalendar().week

# cannot serialize numpy int OR Int64 when writing HDF5 file, so we go for plain int as all NAs are gone now
df_matches['week_number'] = df_matches['week_number'].fillna(0).astype(int)

# add year based on match ISO week
df_matches['year'] = df_matches['match_date'].dt.isocalendar().year.astype(int)

df_matches['week_year'] = df_matches['year'].astype(str) + '-' + df_matches['week_number'].astype(str)

# use a lambda function since isoweek.Week is not vectorized 
df_matches['week_date'] = pd.to_datetime(df_matches.apply(lambda row : Week(int(row["year"]),int(row["week_number"])).monday(),axis=1))



In [14]:
df_matches.query('tournament_id == 59383')

,match_id,replay_id,tournament_id,match_date,match_time,match_conceded,team1_id,team1_coach_id,team1_roster_id,team1_race_name,team1_value,team1_cas_bh,team1_cas_si,team1_cas_rip,team2_id,team2_coach_id,team2_roster_id,team2_race_name,team2_value,team2_cas_bh,team2_cas_si,team2_cas_rip,team1_score,team2_score,team1_win,team2_win,mirror_match,team1_cas,team2_cas,tv_diff,tv_diff2,tv_match,tv_bin,tv_bin2,week_number,year,week_year,week_date
231124,4447417,1606437.0,59383.0,2023-03-27,20:40:35,None,1116987.0,133227.0,4978.0,Underworld Denizens,1000,0.0,2.0,0.0,1118830.0,243311.0,5145.0,Tomb Kings,1150,1.0,0.0,1.0,1.0,0.0,1.0,-1.0,0,2.0,2.0,150,150,1150,1.1M,1.1M,13,2023,2023-13,2023-03-27
231141,4447434,1606445.0,59383.0,2023-03-27,21:40:02,None,1117780.0,220632.0,4975.0,Shambling Undead,1150,0.0,1.0,1.0,1118822.0,131879.0,4979.0,Wood Elf,1150,0.0,0.0,1.0,0.0,3.0,-1.0,1.0,0,2.0,1.0,0,0,1150,1.1M,1.1M,13,2023,2023-13,2023-03-27
231146,4447439,1606455.0,59383.0,2023-03-27,21:48:45,None,1117560.0,28022.0,4969.0,Necromantic Horror,1150,1.0,1.0,1.0,1117426.0,5542.0,4979.0,Wood Elf,1150,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0,3.0,0.0,0,0,1150,1.1M,1.1M,13,2023,2023-13,2023-03-27
231158,4447451,1606462.0,59383.0,2023-03-27,22:14:44,None,1119081.0,27388.0,4963.0,Halfling,760,1.0,1.0,0.0,1118926.0,226515.0,4972.0,Old World Alliance,1150,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0,2.0,3.0,390,390,1150,1.1M,1.1M,13,2023,2023-13,2023-03-27
231184,4447477,1606515.0,59383.0,2023-03-27,23:02:02,None,1117356.0,253629.0,5145.0,Tomb Kings,1150,1.0,0.0,1.0,1118837.0,225446.0,4959.0,Dark Elf,1150,0.0,1.0,0.0,0.0,1.0,-1.0,1.0,0,2.0,1.0,0,0,1150,1.1M,1.1M,13,2023,2023-13,2023-03-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241177,4457470,1618388.0,59383.0,2023-05-07,22:22:54,None,1117894.0,9714.0,4959.0,Dark Elf,1150,1.0,0.0,0.0,1119013.0,247051.0,4975.0,Shambling Undead,1150,0.0,2.0,0.0,2.0,0.0,1.0,-1.0,0,1.0,2.0,0,0,1150,1.1M,1.1M,18,2023,2023-18,2023-05-01
241178,4457471,1618380.0,59383.0,2023-05-07,22:23:16,None,1118910.0,133484.0,4974.0,Orc,1150,0.0,1.0,1.0,1118382.0,246903.0,4966.0,Lizardmen,1150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2.0,0.0,0,0,1150,1.1M,1.1M,18,2023,2023-18,2023-05-01
241180,4457473,1618342.0,59383.0,2023-05-07,22:26:39,None,1117987.0,21930.0,4978.0,Underworld Denizens,770,3.0,2.0,0.0,1118392.0,139015.0,5145.0,Tomb Kings,1150,2.0,1.0,0.0,2.0,1.0,1.0,-1.0,0,5.0,3.0,380,380,1150,1.1M,1.1M,18,2023,2023-18,2023-05-01
241186,4457479,1618379.0,59383.0,2023-05-07,22:47:34,None,1118124.0,136893.0,4958.0,Chaos Renegade,1150,0.0,0.0,0.0,1119002.0,246077.0,4966.0,Lizardmen,1150,0.0,0.0,0.0,0.0,2.0,-1.0,1.0,0,0.0,0.0,0,0,1150,1.1M,1.1M,18,2023,2023-18,2023-05-01


# Step 2: HTML Scraping more match related data

Next, we collect more match related data to add to  `df_matches`.
For example, the **inducements** and **coach rankings**, as well as match performance stats such as total passing distance `pass`. 
This information is not available through the API, but each played match has an associated HTML page at https://fumbbl.com/FUMBBL.php?page=match with more info.

Since we have to fetch the complete HTML page for each match anyways, I decided to split the proces in two steps:

In the first step, the HTML pages for the desired matches are fetched and stored on disk. 
After some optimization fetching 1K matches takes 10 min. So 6K matches per hour.

(Total file size for 154K files is 7GB. These files cannot be stored on Github.)

In the second step, the HTML pages are processed with `BeautifulSoup` to extract inducments and coach rankings.

In [ ]:
# https://thehftguy.com/2020/07/28/making-beautifulsoup-parsing-10-times-faster/

import lxml
import cchardet
from bs4 import BeautifulSoup
import re
import gzip

end_match =  4460944
begin_match = 4429319

print("matches to scrape: ")
n_matches = end_match - begin_match

full_run = 0

print(n_matches)

if(full_run):
    for i in range(n_matches):
        match_id = end_match - i
        api_string = "https://fumbbl.com/FUMBBL.php?page=match&id=" + str(match_id)

        response = requests.get(api_string)

        dirname = "raw/match_html_files/" + str(match_id)[0:4]
        if not os.path.exists(dirname):
            os.makedirs(dirname)

        fname_string = dirname + "/match_" + str(match_id) + ".html.gz"
        
        with gzip.open(fname_string, mode = "wb") as f:
            f.write(response.text.encode("utf-8"))
            f.close()

        if i % 1000 == 0: 
            # write progress report
            print(i, end='')
            print(".", end='')

     

# Step 2b: extract player ids from match data (NOT ACTIVE RIGHT NOW)

Next we write a function that, given a match_id, reads the contents in BeautifulSoup and extracts the stuff we need.
It returns them as separate lists, later to be combined in a pd df.

In [ ]:
def extract_player_ids_from_single_file(match_id):
    
    dirname = "raw/match_html_files/" + str(match_id)[0:4]
    fname_string = dirname + "/match_" + str(match_id) + ".html"  

    with open(fname_string, mode = "r") as f:
        # https://stackoverflow.com/questions/30277109/beautifulsoup-takes-forever-can-this-be-done-faster
        soup = BeautifulSoup(f, 'xml')

    if soup.find("div", {"class": "matchrecord"}) is not None:
        # match record is available
        player_id = []
        player_number = []
        player_name = []

        players = soup.find_all("div", class_="player")

        for p in range(len(players)):
            div = players[p].find("div", class_= "name")
            # https://stackoverflow.com/questions/55442727/remove-unicode-xa0-from-pandas-column
            div_number = players[p].find("div", class_= "number")
            if div_number is not None:
                div_number = div_number.get_text().strip()
                if div_number is not '':
                    player_number.append(div_number)

            for a in div.find_all('a'):
                # url to scrape
                #player_number = p
                player_url = a.get('href') #for getting link
                player_id.append(player_url.split('=', 1)[1])
                # player name
                player_name.append(a.text) #for getting text between the link
        return [match_id] * len(player_id), player_id, player_number, player_name
    else:
        # NOT SMART, this will fuck up the data types (PyTables pickle performance warning)
        return [match_id], [-1], [None], [None]


Profile the workhorse function line by line. Virtually all of the time is now spend in BeautifulSoup, with 70% in the BS parser, and 30% in the find calls that filter out the stuff we need.

In [ ]:
%load_ext line_profiler
%lprun -f extract_player_ids_from_single_file extract_player_ids_from_single_file(4386470)

# 31% of the time is parsing the HTML!!

In [ ]:
extract_player_ids_from_single_file(4353200)

## Extracting the match id - player id list from the match HTML files

This gives us for each match the participating player ids.
Each match contains on average 25 players.
So for 170K matches, we are looking at some 5M player records to create. 
This likely drops again as there will be lots of duplicates, as players are reused.

Processing the HTML for 100 matches takes 24 s, 
Processing the HTML for 1000 matches takes 5 min, 
for 150K we are at 750 min. So roughly 12 hours.
13K 725 min ???

After profiling, now 1000 matches takes 60s and scales (down from 5 min, and increasing due to memory problems).
The trick is to work with lists, and only in the end convert them to to a pandas dataframe.

Expect up to 3 hours

In [ ]:
def extract_player_ids(N):

    target = 'raw/df_player_ids_html_' + time.strftime("%Y%m%d_%H%M%S") + '.h5' # 

    print(target)

    end_match = 4386470 #use from previous cell # 
    begin_match = 4216257 #

    n_matches = end_match - begin_match
    print(n_matches)
    full_run = 1

    #print(n_matches)

    if(full_run):
        match_ids = []
        player_ids = []
        player_numbers = []
        player_names = []
        
        for i in range(n_matches):
            match_id = end_match - i

            # it spends 99% of the time in this function
            match_id_tmp, player_id_tmp, player_number_tmp, player_name_tmp = extract_player_ids_from_single_file(match_id)

            match_ids.extend(match_id_tmp) # use extend instead of + for efficiency
            player_ids.extend(player_id_tmp)
            player_numbers.extend(player_number_tmp)
            player_names.extend(player_name_tmp)

            if i % 1000 == 0: 
            # write progress report
                print(i, end='')
                print(".", end='')

            if i % 1000 == 0:  
            # write data as hdf5 file     
                data = zip(match_ids, player_ids, player_numbers, player_names)

                df_player_ids_html = pd.DataFrame(data, columns = ['match_id', 'player_id', 'player_number', 'player_name'])

                df_player_ids_html.to_hdf(target, key='df_player_ids_html', mode='w')

        # write data as hdf5 file
        data = zip(match_ids, player_ids, player_numbers, player_names)

        df_player_ids_html = pd.DataFrame(data, columns = ['match_id', 'player_id', 'player_number', 'player_name'])
        df_player_ids_html.to_hdf(target, key='df_player_ids_html', mode='w')
    else:
        print("do nothing")
        # read from hdf5 file


#extract_player_ids(-1)

In [ ]:
%load_ext line_profiler
#%lprun -f extract_player_ids extract_player_ids(100)

In [ ]:
df_player_ids_html = pd.read_hdf('raw/df_player_ids_html_20221229_121951.h5')
df_player_ids_html.query('match_id == 4353201') # 4353065

In [ ]:
# remove duplicates
df_player_ids_html[df_player_ids_html.duplicated(['player_id'], keep=False)]

In [ ]:
df_player_ids_html.match_id.nunique()

df_player_ids_html.info()

In [ ]:
df_player_ids_html = df_player_ids_html.drop_duplicates(subset='player_id', keep='first')



In [ ]:
df_player_ids_html

# Processing the match HTML files

I highly recommend [this tutorial](https://hackersandslackers.com/scraping-urls-with-beautifulsoup/) for a great introduction to `BeautifulSoup`.
It allows easy access to the information contained within HTML "div" tags that partition the web page in different sections. 

In addition, to clean up the scraped text, I used the **re** Python module (Regular expressions), part of the [Python standard library](https://docs.python.org/3/library/index.html) to extract the actual inducements from the text string that contains them.

Processing 1000 matches takes 2 min, so to process all 150K matches is expected to take 300 min (in the end it took 800+ min).

Now it takes 1.5 min. 40K matches, expect 1h. check


In [ ]:
def do_all_the_stuff():

    target = 'raw/df_matches_html_' + time.strftime("%Y%m%d_%H%M%S") + '.h5'

    print(target)

    end_match = 4429319 
    begin_match = 4386470

    n_matches = end_match - begin_match
    full_run = 0

    print(n_matches)

    if(full_run):
        match_id = [] # used CTRL + D to add to selection
        team1_inducements = []
        team2_inducements = []
        coach1_ranking = []
        coach2_ranking = []
        team1_comp = []
        team2_comp = []
        team1_pass = []
        team2_pass = []
        team1_rush = []
        team2_rush = []
        team1_block = []
        team2_block = []
        team1_foul = []
        team2_foul = []

        for i in range(n_matches):
            match_id_tmp = end_match - i
            dirname = "raw/match_html_files/" + str(match_id_tmp)[0:4]

            # PM first check gz, if not exist then check for unzipped html
            fname_string = dirname + "/match_" + str(match_id_tmp) + ".html"        
            fname_string_gz = dirname + "/match_" + str(match_id_tmp) + ".html.gz"        

            with gzip.open(fname_string_gz, mode = "rb") as f:
                soup = BeautifulSoup(f, 'xml')

            if soup.find("div", {"class": "matchrecord"}) is not None:
                # match record is available
                inducements = soup.find_all("div", class_="inducements")

                pattern = re.compile(r'\s+Inducements: (.*)\n')

                match = re.match(pattern, inducements[0].get_text())
                if match:
                    team1_inducements_tmp = match.group(1)
                else:
                    team1_inducements_tmp = ''

                match = re.match(pattern, inducements[1].get_text())
                if match:
                    team2_inducements_tmp = match.group(1)
                else:
                    team2_inducements_tmp = ''

                coach_info = soup.find_all("div", class_="coach")
                # grab the ranking
                coach1_ranking_tmp = coach_info[0].get_text()
                coach2_ranking_tmp = coach_info[1].get_text()

                # match performance stats
                div = soup.find_all('div', class_= "player foot")
                # passing completions
                regex = re.compile('.*front comp statbox.*')
                team1_comp_tmp = div[0].find("div", {"class" : regex}).get_text()
                team2_comp_tmp = div[1].find("div", {"class" : regex}).get_text()                   
                # passing distance in yards
                regex = re.compile('.*back pass statbox.*')
                team1_pass_tmp = div[0].find("div", {"class" : regex}).get_text()
                team2_pass_tmp = div[1].find("div", {"class" : regex}).get_text()
                # rushes
                regex = re.compile('.*back rush statbox.*')
                team1_rush_tmp = div[0].find("div", {"class" : regex}).get_text()
                team2_rush_tmp = div[1].find("div", {"class" : regex}).get_text()
                # block
                regex = re.compile('.*back block statbox.*')
                team1_block_tmp = div[0].find("div", {"class" : regex}).get_text()
                team2_block_tmp = div[1].find("div", {"class" : regex}).get_text()
                # foul
                regex = re.compile('.*back foul statbox.*')
                team1_foul_tmp = div[0].find("div", {"class" : regex}).get_text()
                team2_foul_tmp = div[1].find("div", {"class" : regex}).get_text()

                match_id.append(match_id_tmp) # append for single item, extend for multiple items
                team1_inducements.append(team1_inducements_tmp)
                team2_inducements.append(team2_inducements_tmp)
                coach1_ranking.append(coach1_ranking_tmp)
                coach2_ranking.append(coach2_ranking_tmp)
                team1_comp.append(team1_comp_tmp)
                team2_comp.append(team2_comp_tmp)
                team1_pass.append(team1_pass_tmp)
                team2_pass.append(team2_pass_tmp)
                team1_rush.append(team1_rush_tmp)
                team2_rush.append(team2_rush_tmp)
                team1_block.append(team1_block_tmp)
                team2_block.append(team2_block_tmp)
                team1_foul.append(team1_foul_tmp)
                team2_foul.append(team2_foul_tmp)
            
            if i % 1000 == 0: 
            # write progress report
                print(i, end='')
                print(".", end='')

                data = zip(match_id, team1_inducements, team2_inducements, 
                                        coach1_ranking, coach2_ranking, team1_comp, team2_comp,
                                        team1_pass, team2_pass, team1_rush, team2_rush,
                                        team1_block, team2_block, team1_foul, team2_foul)

                df_matches_html = pd.DataFrame(data, columns = ['match_id', 'team1_inducements', 'team2_inducements',
                'coach1_ranking', 'coach2_ranking', 'team1_comp', 'team2_comp',
                'team1_pass', 'team2_pass', 'team1_rush', 'team2_rush',
                'team1_block', 'team2_block', 'team1_foul', 'team2_foul'])

                df_matches_html.to_hdf(target, key='df_matches_html', mode='w')

        # write data as hdf5 file
        data = zip(match_id, team1_inducements, team2_inducements, 
                                coach1_ranking, coach2_ranking, team1_comp, team2_comp,
                                team1_pass, team2_pass, team1_rush, team2_rush,
                                team1_block, team2_block, team1_foul, team2_foul)

        df_matches_html = pd.DataFrame(data, columns = ['match_id', 'team1_inducements', 'team2_inducements',
        'coach1_ranking', 'coach2_ranking', 'team1_comp', 'team2_comp',
        'team1_pass', 'team2_pass', 'team1_rush', 'team2_rush',
        'team1_block', 'team2_block', 'team1_foul', 'team2_foul'])

        df_matches_html.to_hdf(target, key='df_matches_html', mode='w')
    else:
        # read from hdf5 file    
        df_matches_html1 = pd.read_hdf('raw/df_matches_html_20220315_220825.h5') 
        df_matches_html2 = pd.read_hdf('raw/df_matches_html_20220316_133752.h5') 
        df_matches_html3 = pd.read_hdf('raw/df_matches_html_20220608_054453.h5')
        df_matches_html4 = pd.read_hdf('raw/df_matches_html_20230115_133734.h5')
        df_matches_html = pd.concat([df_matches_html1, df_matches_html2, df_matches_html3, df_matches_html4], ignore_index= True)

    return df_matches_html

df_matches_html = do_all_the_stuff()

df_matches_html.info()

In [ ]:
#%load_ext line_profiler
#%lprun -f do_all_the_stuff do_all_the_stuff()
# df_matches_html_20230115_133734


Conclusion of the profiling: >95% of the time is spend within Beautiful Soup. Switch to xml parser doubled processing speed.
No further room for improvements.

## Fix the performance stats

The completions have a few weird values like `4/1`, we drop the slash and the value behind that.
`-` is converted to 0 when the string ends directly after the `-` character, i.e. `-` becomes 0 but `-1` becomes -1.

In [ ]:
df_matches_html['team1_comp'] = df_matches_html['team1_comp'].str.replace(r'(/).*','')
df_matches_html['team1_comp'] = pd.to_numeric(df_matches_html['team1_comp'].str.replace(r'-$','0'))

df_matches_html['team2_comp'] = df_matches_html['team2_comp'].str.replace(r'(/).*','')
df_matches_html['team2_comp'] = pd.to_numeric(df_matches_html['team2_comp'].str.replace(r'-$','0'))

df_matches_html['team1_pass'] = pd.to_numeric(df_matches_html['team1_pass'].str.replace(r'-$','0'))
df_matches_html['team2_pass'] = pd.to_numeric(df_matches_html['team2_pass'].str.replace(r'-$','0'))

df_matches_html['team1_rush'] = pd.to_numeric(df_matches_html['team1_rush'].str.replace(r'-$','0'))
df_matches_html['team2_rush'] = pd.to_numeric(df_matches_html['team2_rush'].str.replace(r'-$','0'))

df_matches_html['team1_block'] = pd.to_numeric(df_matches_html['team1_block'].str.replace(r'-$','0'))
df_matches_html['team2_block'] = pd.to_numeric(df_matches_html['team2_block'].str.replace(r'-$','0'))

df_matches_html['team1_foul'] = pd.to_numeric(df_matches_html['team1_foul'].str.replace(r'-$','0'))
df_matches_html['team2_foul'] = pd.to_numeric(df_matches_html['team2_foul'].str.replace(r'-$','0'))


In [ ]:
df_matches_html.info()

In [ ]:
df_matches_html

## Dataprep: coach rankings

We want to extract the part `CR 149.99` from the scraped coach information field (example `test_string` below). Just as we matches on `Inducements:`, we can match on `CR ` and grab the contents directly after that, stopping when we encounter a whitespace.

We first play around a bit and test until we discover the proper Regular Expression to use :-)


In [ ]:
test_string = 'kingcann Emerging Star CR 149.99 (-0.76)'

pattern = re.compile(r'.*CR (.*)\s\(.*')

match = re.match(pattern, test_string)

if match is not None:
    print(match.group(1)) # group(0) is the whole string
else:
    print("match is none")

test_string

Got 'm! Now that we have figured it out, we can write the code that extracts the coach rankings:

In [ ]:
# Dataprep fix match_id
df_matches_html['match_id'] = pd.to_numeric(df_matches_html.match_id) 

# Dataprep: add the coach rankings as separate cols
df_matches_html['coach1_CR'] = df_matches_html['coach1_ranking'].str.extract(r'.*CR (.*)\s\(.*')
df_matches_html['coach2_CR'] = df_matches_html['coach2_ranking'].str.extract(r'.*CR (.*)\s\(.*')

df_matches_html['coach1_CR'] = pd.to_numeric(df_matches_html['coach1_CR'])
df_matches_html['coach2_CR'] = pd.to_numeric(df_matches_html['coach2_CR'])

# abs
df_matches_html['CR_diff'] = np.abs(df_matches_html['coach1_CR'] - df_matches_html['coach2_CR'])
df_matches_html['CR_diff'] = df_matches_html['CR_diff'].astype(float)

# +/-
df_matches_html['cr_diff2'] = df_matches_html['coach1_CR'] - df_matches_html['coach2_CR']

df_matches_html['cr_diff2_bin'] = pd.cut(df_matches_html['cr_diff2'], bins = [-1*float("inf"), -30, -20, -10, -5, 5, 10, 20, 30, float("inf")], 
 labels=['{-Inf,-30]', '[-30,-20]', '[-20,-10]', '[-10,-5]', '[-5,5]', '[5,10]', '[10,20]', '[20,30]', '[30,Inf]']) 

df_matches_html['coach1_CR_bin'] = pd.cut(df_matches_html['coach1_CR'], 
    bins = [0, 135,145, 155, 165, 175, float("inf")], 
    labels=['CR135-', 'CR140', 'CR150', 'CR160','CR170', 'CR175+'])

df_matches_html['coach2_CR_bin'] = pd.cut(df_matches_html['coach2_CR'], 
    bins = [0, 135,145, 155, 165, 175, float("inf")], 
    labels=['CR135-', 'CR140', 'CR150', 'CR160','CR170', 'CR175+'])


In [ ]:
df_matches_html['coach1_CR_bin'].value_counts()

## Dataprep match inducements for each team

The next trick is to use `pandas` `explode()` method (similar to `separate_rows()` in `tidyverse` R) to give each inducement its own row in the dataset.
This creates a dataframe (`inducements`) similar to `df_mbt` with each match generating at least two rows.


In [ ]:
team1_inducements = df_matches_html[['match_id', 'team1_inducements']]
team2_inducements = df_matches_html[['match_id', 'team2_inducements']]

# make column names equal
team1_inducements.columns = team2_inducements.columns = ['match_id', 'inducements']
team1_inducements['team'] = 'team1'
team2_inducements['team'] = 'team2'

# row bind the two dataframes
inducements = pd.concat([team1_inducements, team2_inducements], ignore_index = True)

# convert comma separated string to list
inducements['inducements'] = inducements['inducements'].str.split(',')

# make each element of the list a separate row
inducements = inducements.explode('inducements')

# strip leading and trailing whitespaces
inducements['inducements'] = inducements['inducements'].str.strip()

# create "star player" label
inducements['star_player'] = 0
inducements.loc[inducements['inducements'].str.contains("Star player"), 'star_player'] = 1

# create "card" label
inducements['special_card'] = 0
inducements.loc[inducements['inducements'].str.contains("Card"), 'special_card'] = 1


In [ ]:
inducements

## Add match HTML data to df_matches

Here we add `df_matches_html` to `df_matches`. This contains each players inducements as a single string, not convenient for analysis.




In [ ]:
df_matches = pd.merge(df_matches, df_matches_html, on='match_id', how='left')

## Add specific inducement info to df_matches

The `inducements` dataframe cannot easily be added to `df_matches`. We can however, extract information from `inducements` at the match level and add this to `df_matches`. Here, I show how to add a 1/0 flag `has_sp` that codes for if a match included any star player.

In [ ]:
df_sp = (inducements
            .groupby("match_id")
            .agg(has_sp = ("star_player", "max"))
            .reset_index()
)


df_matches = pd.merge(df_matches, df_sp, on = "match_id", how = "left")

df_matches['match_id'] = pd.to_numeric(df_matches.match_id) 

# Step 3: API scraping of team data

Great! Almost there. There is still something missing though.

FUMBBL allows coaches to create their own rulesets to play their own leagues and tournaments with. For example, there is a so-called "Secret League" where coaches can play with "Ninja halflings", "Ethereal" spirits etc. Instead of plain normal regular "Halflings" and "Shambling Undead" :-)

Since we want the team strength for the official rulesets BB2016 and BB2020, we need to distinguish those matches from the matches that are played under different rules.
We need to know, for all the matches in our `df_matches` dataset, in what `division` or `league` the match took place, and what version of the rules (encoded in `ruleset`) was used. This information is available in the FUMBBL API, but not on the match level but on the **team** level.

So, let us grab for all teams in `df_matches` the team `division`, `division_id`, `league`, and `ruleset` (last two are both numbers).

As most other available information through the team API can vary over time (i.e. number of games played, how may rerolls a team has, or whether it has an apothecary), we do no fetch this information (expect number of games played), as we cannot link it to specific matches. We only fetch the information we expect to be valid for **all matches** played by this team. 

A limitation of the FUMBBL API is that it shows only the latest version of the teams and leagues data.  

This hides the fact that leagues have changed their rules since they were first created. For example, the NAF used BB2016 rules up until summer of 2021, and thereafter switched to the new BB2020 ruleset for their latest online tournament.
So we have to use our "domain knowledge" here to interpret the data properly.

**PM we now have tournament id  as well, possibly this allows to at least pinpoint when rulesets might have changed**


just get all the teams for local storage.
PM Next time, write code that checks for presence of local team files, and if not present query API.

In [ ]:
# make list of all teams that need to be fetched
#team_ids = list(df_matches['team1_id'].dropna()) + list(df_matches['team2_id'].dropna())

all_team_ids = (df_matches
.loc[:,['team1_id']]
.team1_id.tolist()
) +  (df_matches
.loc[:,['team2_id']]
.team2_id.tolist()
)

# get unique values by converting to a Python set and back to list
all_team_ids = list(set(all_team_ids))

len(all_team_ids)

team_ids_to_fetch = all_team_ids

## first we only get the team data and store it on disk as gzipped JSON

each team takes 1 s, so we expect for 20K teams

4K per hour, 5 hours.
500 min 30K teams. 

75K teams, took 1.5 days

In [ ]:
# save team API call as gzipped JSON object.
# Combine with https://fumbbl.com/api/team/getOptions/1038960

print("teams to scrape: ")
n_teams = len(team_ids_to_fetch)

print(n_teams)

fullrun = 0

if fullrun:
    print('fetching team data for ', len(team_ids_to_fetch), ' teams')

    for t in range(n_teams):
        team_id = int(team_ids_to_fetch[t])
            
        api_string = "https://fumbbl.com/api/team/get/" + str(team_id)

        response = requests.get(api_string)
        response = response.json()

        dirname = "raw/team_files/" + str(team_id)[0:4]
        if not os.path.exists(dirname):
            os.makedirs(dirname)

        fname_string = dirname + "/team_" + str(team_id) + ".json.gz"
        
        with gzip.open(fname_string, mode = "w") as f:
            f.write(json.dumps(response).encode('utf-8'))  
            f.close()

        time.sleep(0.3)
        # also get tournament skills via getoptions
        api_string = "https://fumbbl.com/api/team/getOptions/" + str(team_id)
        response = requests.get(api_string)
        response = response.json()
        response['tournamentSkills'] = json.loads(response['tournamentSkills'])

        fname_string = dirname + "/team_" + str(team_id) + "_skills.json.gz"
        
        with gzip.open(fname_string, mode = "wb") as f:
            f.write(json.dumps(response).encode("utf-8"))
            f.close()
        time.sleep(0.3)

        if t % 1000 == 0: 
            # write progress report
            print(t, end='')
            print(".", end='')


## process team data from disk

So we have to process data for 5K different teams. 

We use the same approach as above, now looping over all `team_id` 's

PM add all relevant team info that add to TV in tournament (rerolls , teamvalue , fan factor etc)

this takes 1 min for 75K teams.

In [ ]:
def do_all_the_team_stuff():
    
    n_teams = len(team_ids_to_fetch)
    #n_teams = 100

    target = 'raw/df_teams_' + time.strftime("%Y%m%d_%H%M%S") + '.h5'
    fullrun = 0

    if fullrun:
        team_id = []
        division_id = []
        division_name = []
        league = []
        ruleset = []
        roster_id = []
        race_name = []
        games_played = []

        print('processing team data for ', len(team_ids_to_fetch), ' teams')

        for t in range(n_teams):    
            team_id_tmp = int(team_ids_to_fetch[t])
            dirname = "raw/team_files/" + str(team_id_tmp)[0:4]

            fname_string_gz = dirname + "/team_" + str(team_id_tmp) + ".json.gz"        
            
            # PM read compressed json file
            with gzip.open(fname_string_gz, mode = "rb") as f:
                team = json.load(f)

            # grab fields
            team_id_tmp = team['id'] # = team_id_tmp
            division_id_tmp = team['divisionId']
            division_name_tmp = team['division']
            ruleset_tmp = team['ruleset']
            league_tmp = team['league']
            roster_id_tmp = team['roster']['id']
            race_name_tmp = team['roster']['name']
            games_played_tmp = team['record']['games']

            team_id.append(team_id_tmp)
            division_id.append(division_id_tmp)
            division_name.append(division_name_tmp)
            league.append(league_tmp)
            ruleset.append(ruleset_tmp)
            roster_id.append(roster_id_tmp)
            race_name.append(race_name_tmp)
            games_played.append(games_played_tmp)       
            
            if t % 5000 == 0: 
                # write tmp data as hdf5 file
                print(t, end='')
                print(".", end='')
                data = zip(team_id, division_id, division_name, league, ruleset, roster_id, race_name,  games_played)

                df_teams = pd.DataFrame(data, columns=['team_id', 'division_id', 'division_name',  'league' ,
        'ruleset', 'roster_id', 'race_name',  'games_played'])
                df_teams.to_hdf(target, key='df_teams', mode='w')
        
        data = zip(team_id, division_id, division_name, league, ruleset, roster_id, race_name,  games_played)
                
        df_teams = pd.DataFrame(data, columns=['team_id', 'division_id', 'division_name',  'league' ,
        'ruleset', 'roster_id', 'race_name',  'games_played'])
        df_teams.to_hdf(target, key='df_teams', mode='w')

    else:
        # read from hdf5 file
        #df_teams1 = pd.read_hdf('raw/df_teams_20220316_221902.h5')
        #df_teams2 = pd.read_hdf('raw/df_teams_20220609_062756.h5')
        df_teams = pd.read_hdf('raw/df_teams_20230122_091040.h5')
        #df_teams = pd.concat([df_teams1, df_teams2], ignore_index=True)
    return df_teams

df_teams = do_all_the_team_stuff()

df_teams['roster_name'] = df_teams['roster_id'].astype(str) + '_' + df_teams['race_name']

df_teams.shape

df_teams.info()

    

In [ ]:
df_teams

## Dataprep: Add ruleset_version and division_name to df_teams

Lets have look at the various divisions and leagues, and which rulesets are used.
There are a lot of small leagues being played on FUMBBL, they account for maybe X% of all the matches.

We only look at divisions and leagues with a sufficient volume of matches, or otherwise we do not have sufficient statistics for each race.

So I aggregated the data by division, league and ruleset, and filtered on at least 150 different teams that have played at least once last year.
Apart from the main "Divisions" that are part of FUMBBL, there were a few user-run leagues present in this table, so I looked up their names on FUMBBL and what ruleset is used (BB2016, BB2020 or some other variant). This information (contained in an xlsx) is added to the dataset below.


In [ ]:
# add ruleset_version and division_name from codelist CSV
ruleset_division_names = pd.read_csv('codelists/ruleset_division_names.csv')

# initial creation of the codelist
#ruleset_division_names.to_csv('codelists/ruleset_division_names.csv', encoding='utf-8', index=False)


df_teams = pd.merge(df_teams, ruleset_division_names, on= ['league', 'ruleset', 'division_id'], how='left')

df_teams['division_name'] = df_teams['new_division_name']

df_teams = df_teams.drop('new_division_name', 1)

df_teams['division_id'] = pd.to_numeric(df_teams.division_id) 
df_teams['roster_id'] = pd.to_numeric(df_teams.roster_id) 
df_teams['team_id'] = pd.to_numeric(df_teams.team_id) 
df_teams['games_played'] = pd.to_numeric(df_teams.games_played) 

df_teams['league'] = pd.to_numeric(df_teams.league) 
df_teams['ruleset'] = pd.to_numeric(df_teams.ruleset) 


In [ ]:
ruleset_division_names.info()

In [ ]:
df_teams = df_teams.drop(['group_id', 'details', 'tv','type'], 1)

Check if we have labeled all high volume leagues, divisions and rulesets:

**PM we see that (NAF) matches played previously under ruleset 2228 are now labeled as ruleset 2310?
this has a few changes (tier, gold, crossleague)
Do we also see this in the XML API**

So a group / league has a current ruleset attached.
This current ruleset is also presented / attached to the team.
Is it also attached to a match? can we know under which ruleset a match played?

matches are played within a tournament id. within a group.
so there is a link between tournament and ruleset

In [ ]:
(df_teams
    .groupby(['ruleset', 'league', 'division_id', 'division_name',  'ruleset_version'], dropna=False)
    .agg( n_teams = ('ruleset', 'count')
    )
    .sort_values('n_teams', ascending = False)
    .query('n_teams > 150')['n_teams']
    .reset_index()
)

## Dataprep: Merging the division/ruleset data with the match data

As most of the data in `df_teams` is actually on the level of the match, we can merge on `team1_id` after leaving out the team specific variables.

In [ ]:
df_matches = pd.merge(df_matches, df_teams.drop(['race_name', 'roster_id', 'roster_name', 'games_played'], 1), left_on='team1_id', right_on = 'team_id', how='left')

df_matches['team1_id'] = pd.to_numeric(df_matches.team1_id) 
df_matches = df_matches.drop('team_id', 1)

# Step 4: Create matches by team DataFrame

When analyzing the data, we also like to have a dataframe `df_mbt (df_matches_by_team)` that contains, for each match, a separate row for each team participating in that match.
This structure is nicely visualized [at the Nufflytics blog](https://www.nufflytics.com/post/the-value-of-tv/).
Such a dataset is suitable for adding, at the match level, data that is specific for each team - coach pair, such as team value, coach rating etc.
For example, we can imagine adding more team level data, such as casualties caused during the match, or team composition at the start of the match etc.


In [ ]:
# make two copies, one for each team in the match
team1_data = df_matches[['match_id', 'match_date', 'week_number',	'year',	'week_year', 'week_date', 'team1_id', 'ruleset', 'league', 'division_id', 'division_name', 'ruleset_version',
    'team1_coach_id', 'team1_race_name', 'team1_value', 'team1_score', 'team1_win', 'tv_diff', 'tv_match', 'team2_coach_id', 'team2_race_name', 'team2_value', 'team2_score', 
    'team1_comp', 'team1_pass', 'team1_rush', 'team1_block', 'team1_foul', 'team1_cas', 'team1_cas_bh', 'team1_cas_si', 'team1_cas_rip',
    'team2_comp', 'team2_pass', 'team2_rush', 'team2_block', 'team2_foul', 'team2_cas', 'team2_cas_bh', 'team2_cas_si', 'team2_cas_rip',
    'tv_bin', 'tv_bin2', 'mirror_match', 'coach1_CR', 'coach2_CR', 'coach1_CR_bin', 'CR_diff',  'has_sp']].copy()

team2_data = df_matches[['match_id', 'match_date', 'week_number',	'year',	'week_year', 'week_date', 'team2_id', 'ruleset', 'league', 'division_id', 'division_name', 'ruleset_version',
    'team2_coach_id', 'team2_race_name', 'team2_value', 'team2_score', 'team2_win', 'tv_diff', 'tv_match', 'team1_coach_id', 'team1_race_name', 'team1_value', 'team1_score',
    'team2_comp', 'team2_pass', 'team2_rush', 'team2_block', 'team2_foul', 'team2_cas', 'team2_cas_bh', 'team2_cas_si', 'team2_cas_rip',
    'team1_comp', 'team1_pass', 'team1_rush', 'team1_block', 'team1_foul', 'team1_cas', 'team1_cas_bh', 'team1_cas_si', 'team1_cas_rip',
    'tv_bin', 'tv_bin2', 'mirror_match', 'coach2_CR', 'coach1_CR', 'coach2_CR_bin','CR_diff', 'has_sp']].copy()

team1_data.columns = team2_data.columns = ['match_id', 'match_date', 'week_number',	'year',	'week_year', 'week_date', 'team_id', 'ruleset', 'league', 'division_id', 'division_name', 'ruleset_version',
    'coach_id', 'race_name', 'team_value', 'team_score', 'wins', 'tv_diff', 'tv_match',  'away_coach_id', 'away_race_name', 'away_team_value', 'away_team_score',
    'home_comp', 'home_pass', 'home_rush', 'home_block', 'home_foul',  'home_cas', 'home_cas_bh', 'home_cas_si', 'home_cas_rip',
    'away_comp', 'away_pass', 'away_rush', 'away_block', 'away_foul',  'away_cas', 'away_cas_bh', 'away_cas_si', 'away_cas_rip',
    'tv_bin', 'tv_bin2', 'mirror_match', 'coach_CR', 'away_coach_CR', 'coach_CR_bin','CR_diff', 'has_sp']

# combine both dataframes
df_mbt = pd.concat([team1_data, team2_data])

df_mbt['tv_diff2'] = df_mbt['team_value'] - df_mbt['away_team_value']
df_mbt['cr_diff2'] = df_mbt['coach_CR'] - df_mbt['away_coach_CR']

In [ ]:
# add games played for each team

df_mbt = pd.merge(df_mbt, df_teams[['team_id', 'games_played']], left_on='team_id', right_on = 'team_id', how='left')

## Adding outcome weights

One way to measure team strength is to calculate a win rate.
If we want to calculate win rates, we need to decide how to weigh a draw.
In Blood Bowl data analysis, it seems that a 2:1:0 (W / D / L) weighting scheme is most commonly used. 
So if we want to compare with others, it makes sense to adapt this scheme as well.
If we divide these weights by two we get something that, if we average it, we can interpret as a win rate.

This scheme has the advantage that the weighted average win percentage over all matches is always 50%, creating a nice reference point allowing conclusions such as "this and that team has an x percent above average win percentage"

In [ ]:
df_mbt.loc[df_mbt['wins'] == 0, 'wins'] = 0.5
df_mbt.loc[df_mbt['wins'] == -1, 'wins'] = 0

# convert to float
df_mbt['wins'] = df_mbt['wins'].astype(float)

At this point, Lets have a look at our dataset again:

In [ ]:
df_mbt.query("coach_id == 255851").sort_values('match_date')


# Step 5: adding race classifications to df_mbt (team tiers, bash/dash/hybrid)

There are team classifications for strength (tiers) and classifications that focus on play style (Bash/Dash/Hybrid).
The Bash/Dash/Hybrid classification i use is based on this great [Nufflytics blog](https://www.nufflytics.com/post/bash-dash-hybrid-by-the-numbers/), and my own subjective choices.
Here we add both to the `df_mbt` dataframe, as in this dataset each row is about a single team, instead of a pairing.

According to [this article from the NAF from 2017](https://www.thenaf.net/2017/05/tiers/), already since 2010 efforts were made to balance things out a bit between the different team strengths. For example, the weaker teams get more gold to spend on players, or get more so-called "Star player points" to spend on skilling players up. 

According to [the NAF](https://www.thenaf.net/tournaments/information/tiers-and-tiering/), traditionally team tiering consists of three groups, with Tier 1 being the strongest teams, and tier 3 the weakest teams. 

The GW BB2020 rule book also contains three tier groups, that are similar to the NAF tiers: except for Humans and Old World Alliance. 

And in november 2021, Games Workshop published an update of the three tier groups, now with High Elves moving from tier 2 to tier 1, and Old World Alliance moving back to tier 2.

Finally, I added an additional race classification into the well known bash/dash (agile)/hybrid / stunty classes.

**PM Vampires is on other**


In [ ]:
race_tiers = pd.read_csv('codelists/race_tiers_mapping.csv')
#race_tiers = race_tiers[ ['race_name', 'bb2020_tier', 'naf_tier', 'bb2020_nov21_tier', 'race_type']]
#race_tiers = race_tiers.dropna()

In [ ]:
race_tiers

In [ ]:
# add bb2020 tiers
df_mbt = pd.merge(df_mbt, race_tiers, on='race_name', how='left')



# Step 2b: API scraping the player data: df_players

**PM NOT USED, WHY HERE**


Now that we have for all matches the player_id's involved, we fetch the player data.
Were mostly interested in the player position, so we can reconstruct the team roster for a specific match.
We can also analyze the development of teams over time: which positionals are bought in what order?

1K players takes 10 min, so 4K players 40 min.

We use the API, as it contains identical info as the HTML page.

https://fumbbl.com/api/player/get/13524599

PM Need to add the match_id 
so we are facing 1M players to scrape.
at a rate of 2 scrapes per second, this will take 6 days of permanent scraping.
We'll subset on BB2020, this will take 3 days.
170K matches takes 8gb of disk space.
check filesize of a single player json object. Maybe compress?


In [ ]:
import json

def write_json_file(json_object, player_id):

    dirname = "raw/player_html_files/" + str(player_id)[0:4]
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    fname_string = dirname + "/player_" + str(player_id) + ".html"

    with open(fname_string, mode = "w", encoding='UTF-8') as f:
        f.write(json.dumps(json_object, ensure_ascii=False, indent=4))
        

In [ ]:
player_id = 14693304

api_string = "https://fumbbl.com/api/player/get/" + str(player_id)

player = requests.get(api_string)
# PM here save JSON as file
player = player.json()

write_json_file(player, player_id)

In [ ]:
# df_player_ids_html needed in bg
def fetch_player_data_from_api(n_players):
    full_run = 1
    
    if n_players > len(df_player_ids_html):
        n_players = len(df_player_ids_html)
    print(n_players)

    for i in range(n_players):
        player_id = df_player_ids_html.iloc[i].player_id
        api_string = "https://fumbbl.com/api/player/get/" + str(player_id)
        # PM add exception handling
        player = requests.get(api_string)
        # PM here save JSON as file
        player = player.json()
        write_json_file(player, player_id)
        
        if i % 100 == 0: 
            #show progress
            print(i, end='')
            print(".", end='')

    return print(i+1, "files written")

fetch_player_data_from_api(10)


## Processing the player JSON data

In [ ]:
import json

def read_json_file(player_id):

    dirname = "raw/player_html_files/" + str(player_id)[0:4]
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    fname_string = dirname + "/player_" + str(player_id) + ".html"

    with open(fname_string, mode = "r", encoding='UTF-8') as f:
        json_object = json.load(f)

    return json_object

player = read_json_file(14693304)
player['name']

In [ ]:
player

Need to fetch the player position to check what skills are added.
https://fumbbl.com/api/position/get/39328
But this is the current set of skills, not necessary what skills were used in a particular game.

For now we just go for position plots for say team values around 1200K.
This will not work because the 1200K includes skills and we do not know them.

Maybe check for tournaments with fixed skill choice?

In [ ]:
def process_player_data_from_file(n_players):
    target = 'raw/df_players_' + 'tst' + '.h5' # time.strftime("%Y%m%d_%H%M%S") 
    print(target)

    full_run = 1
    
    if n_players > len(df_player_ids_html):
        n_players = len(df_player_ids_html)
    print(n_players)

    if(full_run):
        player_ids = []
        team_id = []
        status = []
        number = []
        name = []
        position_id = []
        position_name = []
        stats_ma = []
        stats_st = []
        stats_ag = []
        stats_pa = []
        stats_av = []

        for i in range(n_players):
            player_id = df_player_ids_html.iloc[i].player_id

            player = read_json_file(player_id)
            
            if player: # fix for matches that do not exist
                player_ids.append(player['id'])
                team_id.append(player['teamId'])
                status.append(player['status'])
                number.append(player['number'])
                name.append(player['name'])
                position_id.append(player['position']['id'])
                position_name.append(player['position']['name'])
                stats_ma.append(player['stats']['ma'])
                stats_st.append(player['stats']['st'])
                stats_ag.append(player['stats']['ag'])
                stats_pa.append(player['stats']['pa'])
                stats_av.append(player['stats']['av'])
                # PM skills, injuries: add in for loop (variable length)

            else:
                # empty data for this match, create empty row
                print('An error has occurred.')
                #df_players.loc[i] = np.repeat([np.NaN], 12, axis=0)
                #df_players.loc[i]['player_id'] = int(player_id)
            
            if i % 100 == 0: 
                # write tmp data as hdf5 file
                print(i, end='')
                print(".", end='')
                
                data = zip(player_ids, team_id, status, number, name, 
                position_id, position_name,
                stats_ma, stats_st, stats_ag, stats_pa,  stats_av)

                df_players = pd.DataFrame(data, columns = ['player_id', 'team_id', 'status', 'number', 'name', 
                'position_id', 'position_name',
                'stats_ma', 'stats_st', 'stats_ag', 'stats_pa',  'stats_av'
                ])
                df_players.to_hdf(target, key='df_players', mode='w')

        # write data as hdf5 file
        data = zip(player_ids, team_id, status, number, name, 
                position_id, position_name,
                stats_ma, stats_st, stats_ag, stats_pa,  stats_av)

        df_players = pd.DataFrame(data, columns = ['player_id', 'team_id', 'status', 'number', 'name', 
                'position_id', 'position_name',
                'stats_ma', 'stats_st', 'stats_ag', 'stats_pa',  'stats_av'
                ])
        df_players.to_hdf(target, key='df_players', mode='w')
    else:
        # read from hdf5 file
        print("do nothing")
    return df_players

df_players = process_player_data_from_file(10)
df_players = df_players.sort_values(by=['team_id']).reset_index(drop=True)
df_players.shape

In [ ]:
df_players

# Step 6 add tournament info

create list of all tournament ids.
Fetch and store as JSON.
Then process and add tournament names to `df_matches`.

In [ ]:
# make list of all teams that need to be fetched
#team_ids = list(df_matches['team1_id'].dropna()) + list(df_matches['team2_id'].dropna())

tournament_ids = df_matches['tournament_id'].values

# get unique values by converting to a Python set and back to list
tournament_ids = list(set(tournament_ids))

len(tournament_ids)



In [ ]:
# save tournament API call as gzipped JSON object.

print("tournaments to fetch: ")

n_tourneys = len(tournament_ids)

print(n_tourneys)

fullrun = 0

if fullrun:
    print('fetching tournament data for ', n_tourneys, ' tourneys')

    for t in range(n_tourneys):
        tournament_id = int(tournament_ids[t])
            
        api_string = "https://fumbbl.com/api/tournament/get/" + str(tournament_id)

        response = requests.get(api_string)
        response = response.json()

        dirname = "raw/tournament_files/" + str(tournament_id)[0:2]
        if not os.path.exists(dirname):
            os.makedirs(dirname)

        fname_string = dirname + "/tournament_" + str(tournament_id) + ".json.gz"
        
        with gzip.open(fname_string, mode = "w") as f:
            f.write(json.dumps(response).encode('utf-8'))  
            f.close()

        time.sleep(0.3)

        if t % 100 == 0: 
            # write progress report
            print(t, end='')
            print(".", end='')

## process the tourney json files from disk

In [ ]:
def do_all_the_tourney_stuff():
    
    n_tourneys = len(tournament_ids)
    #n_tourneys = 100

    target = 'raw/df_tourneys_' + time.strftime("%Y%m%d_%H%M%S") + '.h5'
    fullrun = 0

    if fullrun:
        tournament_id = []
        group_id = []
        tournament_type = []
        tournament_progression = []
        tournament_name = []
        tournament_start = []
        tournament_end = []

        print('processing tourney data for ', n_tourneys, ' teams')

        for t in range(n_tourneys):    
            tournament_id_tmp = int(tournament_ids[t])
            dirname = "raw/tournament_files/" + str(tournament_id_tmp)[0:2]

            fname_string_gz = dirname + "/tournament_" + str(tournament_id_tmp) + ".json.gz"        
            
            # PM read compressed json file
            with gzip.open(fname_string_gz, mode = "rb") as f:
                tournament = json.load(f)

            if str(tournament) != 'No such tournament.':
                # grab fields
                tournament_id_tmp = tournament['id'] # = tournament_id_tmp
                group_id_tmp = tournament['group']
                tournament_type_tmp = tournament['type']
                tournament_progression_tmp = tournament['progression']
                tournament_name_tmp = tournament['name']
                tournament_start_tmp = tournament['start']
                tournament_end_tmp = tournament['end']

                tournament_id.append(tournament_id_tmp)
                group_id.append(group_id_tmp)
                tournament_type.append(tournament_type_tmp)
                tournament_progression.append(tournament_progression_tmp)
                tournament_name.append(tournament_name_tmp)
                tournament_start.append(tournament_start_tmp)
                tournament_end.append(tournament_end_tmp)   

            if t % 500 == 0: 
                # write tmp data as hdf5 file
                print(t, end='')
                print(".", end='')
                data = zip(tournament_id, group_id, tournament_type, tournament_progression, tournament_name, tournament_start, tournament_end)

                df_tourneys = pd.DataFrame(data, columns=['tournament_id', 'group_id', 'tournament_type', 'tournament_progression', 
                'tournament_name', 'tournament_start', 'tournament_end'])
                df_tourneys.to_hdf(target, key='df_tourneys', mode='w')

        data = zip(tournament_id, group_id, tournament_type, tournament_progression, tournament_name, tournament_start, tournament_end)
        df_tourneys = pd.DataFrame(data, columns=['tournament_id', 'group_id', 'tournament_type', 'tournament_progression', 
        'tournament_name', 'tournament_start', 'tournament_end'])
        df_tourneys.to_hdf(target, key='df_tourneys', mode='w')   


    else:
        # read from hdf5 file
        df_tourneys = pd.read_hdf('raw/df_tourneys_20230122_135939.h5')

    return df_tourneys

df_tourneys = do_all_the_tourney_stuff()

df_tourneys.shape

df_tourneys['tournament_id'] = pd.to_numeric(df_tourneys.tournament_id) 
df_tourneys['group_id'] = pd.to_numeric(df_tourneys.group_id) 
df_tourneys['tournament_type'] = pd.to_numeric(df_tourneys.tournament_type) 

df_tourneys.info()

In [ ]:
df_tourneys

PM next time also fetch ruleset JSON files

## add tournament name to match data

include tournament tyep
https://fumbbl.com/help:Tournament+Structure

In [ ]:
df_matches.iloc[[0]]

In [ ]:
df_matches = pd.merge(df_matches, df_tourneys[['tournament_id', 'tournament_name', 'tournament_type']], left_on='tournament_id', right_on = 'tournament_id', how='left')



In [ ]:
df_matches.query("match_id == 4421729")

# Step 7 grab Skill code list

In [ ]:
%run read_json_file.py
%run write_json_file.py

In [ ]:
api_string = "https://fumbbl.com/api/skill/list"

skill_list = requests.get(api_string)
skill_list = skill_list.json()

write_json_file(skill_list, fname = "raw/skill_list_bb2016.json")

In [ ]:
api_string = "https://fumbbl.com/api/skill/list/2020"

skill_list = requests.get(api_string)
skill_list = skill_list.json()

write_json_file(skill_list, fname = "raw/skill_list_bb2020.json")

Convert skill code JSONs into a single pandas dataframe 

In [ ]:
skill_list = read_json_file(fname = "raw/skill_list_bb2020.json")
skill_list = pd.json_normalize(skill_list)
skill_list.rename({'id' : 'skill_id'}, axis = 1, inplace=True)

skill_list = skill_list.loc[:, ['skill_id', 'name']]

skill_list.head(5)

In [ ]:
skill_list2 = read_json_file(fname = "raw/skill_list_bb2016.json")
skill_list2 = pd.json_normalize(skill_list2)
skill_list2.rename({'id' : 'skill_id'}, axis = 1, inplace=True)

skill_list2 = skill_list2.loc[:, ['skill_id', 'name']]

skill_list2.head(5)

In [ ]:
df_skills = pd.concat([skill_list, skill_list2])

In [ ]:
df_skills.sort_values('skill_id')

# Save all prepped datasets as HDF5 and CSV files

HD5 files can be read by both Python and [R](https://cran.r-project.org/web/packages/hdf5r/index.html) and preserve column data types.

**Update** the HDF5 schema used by pandas is highly specific to pandas, it is not designed for external use. An R code snippet is available but does not work with table format. For now using the CSVs for R is advised

CSV files are the lingua franca across all data analysis software.

A dataset release consists of three datasets:
* A list of matches, identified by match_id
* A list of matches by team, identified by match_id and team_id
* A list of inducements by match_id

In [ ]:
# use this to locate variables that cannot be serialized by hdf5
#df_matches.loc[:, :'week_date'].info()

In [ ]:
target = 'datasets/current/df_matches'

df_matches.to_hdf(target + '.h5', key='df_matches', mode='w', format = 't',  complevel = 9)
df_matches.to_csv(target + '.csv')


In [ ]:
target = 'datasets/current/df_mbt'

df_mbt.to_hdf(target + '.h5', key='df_mbt', mode='w', format = 't',  complevel = 9)
df_mbt.to_csv(target + '.csv')


In [ ]:
target = 'datasets/current/inducements'
inducements.to_hdf(target + '.h5', key='inducements', mode='w', format = 't',  complevel = 9)
inducements.to_csv(target + '.csv')

In [ ]:
target = 'datasets/current/df_tourneys'

df_tourneys.to_hdf(target + '.h5', key='df_tourneys', mode='w', format = 't',  complevel = 9)
df_tourneys.to_csv(target + '.csv')

In [ ]:
target = 'datasets/current/df_skills'

df_skills.to_hdf(target + '.h5', key='df_skills', mode='w', format = 't',  complevel = 9)
df_skills.to_csv(target + '.csv')

# Choosing a license for the public dataset

An important part of making data publicly available is being explicit about what is allowed if people want to use the dataset.
However, before we do so, we have to check if **we** are actually allowed to publish the data. This is explained nicely [in a blogpost by Elizabeth Wickes](https://datacarpentry.org/blog/2016/06/data-licensing).

Since our data comes from the **FUMBBL.com** website, we check the [**Privacy policy**](https://fumbbl.com/p/privacy) where all users, including myself have agreed on when signing up. It contains this part which is specific to the unauthenticated API, which we use to fetch the data, as well as additional public match data, such as which inducements are used in a match, and the Coach rankings of the playing coaches that were current when the match was played.

```
Content you provide through the website
All the information you provide through the website is processed by FUMBBL. This includes things such as forum posts, private message posts, blog entries, team and player names and biographies and news comments. Data provided this way is visible by other people on the website and in most cases public even to individuals without accounts (not including private messages), and as such are considered of public interest. If direct personal information is posted in public view, you can contact moderators to resolve this. Match records are also considered content in this context, and is also considered of public interest. This data is collected as the primary purpose of the website and it is of course entirely up to you how much of this is provided to FUMBBL. 

Third party sharing
Some of the public data is available through a public (*i.e. unauthenticated*) API, which shares some of the information provided by FUMBBL users in a way suitable for third-party websites and services to process.

The data available through the unauthenticated API is considered non-personal as it only reflects information that is public by its nature on the website. The authenticated API will only show information connected to the authenticated account.
```

I conclude that since the match data is already considered public content, there is no harm in collecting this public data in a structured dataset and placing this data in a public repository. I also verified this with Christer, the site owner. 


The final step is then to decide what others are allowed to do with this data. In practice, this means choosing a license under which to release the dataset. I decided to choose a CC0 license: this places the data in the public domain, and people can use the dataset as they wish. Citing or mentioning the source of the data would still be appreciated of course.

# List of possible future improvements

* Scraping the players (only most recent version, so no player development history)
* Scraping the rulesets (for example to identify resurrection tournaments where players choose skills and use tiers)
* Switch to feather or Parquet dataformat
* catch exception: **PM we cannot deal yet with the situation HTTPSConnectionPool(host='fumbbl.com', port=443): Max retries exceeded with url: /api/match/get/4221820 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f4acff12be0>: Failed to establish a new connection: [Errno 110] Connection timed out',))**
* PM we now have tournament id  as well, possibly this allows to at least pinpoint when rulesets might have changed
* PM we see that (NAF) matches played previously under ruleset 2228 are now labeled as ruleset 2310? this has a few changes (tier, gold, crossleague)
* Do we also see this in the XML API
* cr_bin variable is gone?
